<a href="https://colab.research.google.com/github/srilamaiti/spring_2023_w266_final_project_heesuk_iris_srila/blob/main/iris/IL_data_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/competitions/feedback-prize-english-language-learning

In [2]:
# install
!pip install kaggle
!pip install transformers
!pip install ktrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 81.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 65.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.4/265.4 KB 29.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.9 MB/s eta 0:00:00
  Preparing metadat

In [3]:
import pandas as pd
import numpy as np
import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import tensorflow as tf
import os

import math
from typing import Optional
import transformers
from ktrain import text

import torch
import torch.utils.checkpoint as checkpoint
import tqdm

In [4]:
# Import required libraries/code
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

from torch import nn
from torchvision import transforms, datasets

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

[INFO] Couldn't find torchinfo... installing it.
[INFO] Couldn't find going_modular scripts... downloading them from GitHub.
Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 3578, done.
remote: Counting objects: 100% (200/200), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 3578 (delta 88), reused 189 (delta 82), pack-reused 3378
Receiving objects: 100% (3578/3578), 647.31 MiB | 19.64 MiB/s, done.
Resolving deltas: 100% (2044/2044), done.
Updating files: 100% (240/240), done.


In [5]:
# data
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

%cd "gdrive/MyDrive/Colab Notebooks/"

Mounted at /content/gdrive
/content/gdrive/MyDrive/Colab Notebooks


In [ ]:
## only needed to run it once to download

# os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Colab Notebooks/"

# !kaggle competitions download -c feedback-prize-english-language-learning 
# !unzip -q feedback-prize-english-language-learning.zip -d .

In [6]:
train = pd.read_csv("train.csv")
print(f"train shape: {train.shape}")
train.head()

train shape: (3911, 8)


,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [7]:
test = pd.read_csv("test.csv")
print(f"test shape: {test.shape}")
test.head()

test shape: (3, 2)


,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


In [9]:
# # histogram with cohesion, syntax, vocabulary, phraseology, grammar, conventions 
# for y in ["cohesion","syntax","vocabulary", "phraseology","grammar", "conventions"]:
#   print(train[y].value_counts().sort_index())
#   print()
#   train[y].hist()
#   plt.show()

In [10]:
retribert_model_checkpoints = "yjernite/retribert-base-uncased"
retribert_model = transformers.BertModel.from_pretrained(retribert_model_checkpoints)
# retribert_model

# Freeze all base layers in the "features" section of the model (the feature extractor) by setting requires_grad=False
for param in retribert_model.parameters():
    param.requires_grad = False

retribert_tokenizer = transformers.BertTokenizer.from_pretrained(retribert_model_checkpoints)

In [11]:
retri_cuda = retribert_model.cuda()

In [13]:
def set_config_param(seed = 99):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    tf.keras.backend.clear_session()
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Kaggle"
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_columns', None)

set_config_param()

In [14]:
input_train_df = train
input_test_df = test
label_cols = input_train_df.columns[2:]
orig_train_df = copy.deepcopy(input_train_df)
orig_train_df.head()
     
shuffle = np.random.permutation(np.arange(orig_train_df.shape[0]))
orig_train_df = orig_train_df.iloc[shuffle]
split=(0.8,0.1,0.1)
splits = np.multiply(len(orig_train_df), split).astype(int)
df_train, df_val, df_test = np.split(orig_train_df, [splits[0], splits[0] + splits[1]])

X_train, X_val, X_test = df_train['full_text'], df_val['full_text'], df_test['full_text']
y_train, y_val, y_test = np.array(df_train[label_cols]), np.array(df_val[label_cols]), np.array(df_test[label_cols])

In [15]:
MAX_LENGTH = 512
epochs = 5
batch_size = 8
dropout = .1
learning_rate = .00005
number_of_hidden_layer = 1
hidden_layer_node_count = 256
trainable_flag = False
retrain_layer_count = 0

In [16]:
def MCRMSE(y_true, y_pred):
    colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=1)
    return tf.reduce_mean(tf.sqrt(colwise_mse), axis=-1, keepdims=True)

def plot_loss_accuracy(history, col_list):
    fig, ax = plt.subplots(2, 6, figsize=(16, 6), sharex='col', sharey='row')
    fig.tight_layout(pad=5.0)
    for idx, col in enumerate(col_list):

        ax[0, idx].plot(history[col + '_loss'], lw=2, color='darkgoldenrod')
        ax[0, idx].plot(history['val_' + col + '_loss'], lw=2, color='indianred')
        #ax[0, idx].legend(loc='center left')
        ax[0, idx].legend(['Train', 'Validation'], fontsize=5)
        ax[0, idx].set_xlabel('Epochs', size=10)
        ax[0, idx].set_title('Loss: ' + col)

        ax[1, idx].plot(history[col + '_accuracy'], lw=2, color='darkgoldenrod')
        ax[1, idx].plot(history['val_' + col + '_accuracy'], lw=2, color='indianred')
        #ax[0, idx].legend(loc='center left')
        ax[1, idx].legend(['Train', 'Validation'], fontsize=5)
        ax[1, idx].set_xlabel('Epochs', size=10)
        ax[1, idx].set_title('Accuracy: ' + col)

def encode_text(text, tokenizer):
    
    encoded = tokenizer.batch_encode_plus(
        text,
        add_special_tokens=True,
        max_length=MAX_LENGTH,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors="tf",
    )

    input_ids = np.array(encoded["input_ids"], dtype="int32")
    attention_masks = np.array(encoded["attention_mask"], dtype="int32")

    return {
        "input_ids": input_ids,
        "attention_masks": attention_masks
    }

In [17]:
train_encodings = encode_text(df_train['full_text'].tolist(), retribert_tokenizer)
val_encodings = encode_text(df_val['full_text'].tolist(), retribert_tokenizer)
test_encodings = encode_text(df_test['full_text'].tolist(), retribert_tokenizer)

In [ ]:
# double check encodings and length=512

In [ ]:
#### from
# https://stackoverflow.com/questions/55369821/how-to-train-a-neural-network-model-with-bert-embeddings-instead-of-static-embed

X_train = [retribert_tokenizer.tokenize('[CLS] ' + sent + ' [SEP]') for sent in X_train] 
X_train_tokens = [retribert_tokenizer.convert_tokens_to_ids(sent) for sent in X_train]
# # X_train[0]
# # X_train_tokens[0]
X_test = [retribert_tokenizer.tokenize('[CLS] ' + sent + ' [SEP]') for sent in X_test] 
X_test_tokens = [retribert_tokenizer.convert_tokens_to_ids(sent) for sent in X_test]
# y_train_tensor = torch.from_numpy(y_train)

In [ ]:
### train individual tensors
y_train_cohesion = [y_train[i][0] for i in range(len(y_train))]
y_train_syntax = [y_train[i][1] for i in range(len(y_train))]
y_train_vocabulary = [y_train[i][2] for i in range(len(y_train))]
y_train_phraseology = [y_train[i][3] for i in range(len(y_train))]
y_train_grammar = [y_train[i][4] for i in range(len(y_train))]
y_train_convention = [y_train[i][5] for i in range(len(y_train))]

y_train_cohesion_tensor = torch.Tensor(y_train_cohesion)

In [18]:
# https://stackoverflow.com/questions/64156202/add-dense-layer-on-top-of-huggingface-bert-model
class CustomBERTModel(nn.Module):
    def __init__(self):
          super(CustomBERTModel, self).__init__()
          self.retribert = transformers.BertModel.from_pretrained(retribert_model_checkpoints)
          # for param in self.retribert.parameters():
            # param.requires_grad = False
          self.linear1 = nn.Linear(768, 9)
          self.dropout1 = nn.Dropout(0.1)

    def forward(self, ids, mask):
          outputs = self.retribert(ids,attention_mask=mask)
          last_hidden_state = outputs[0]

          # sequence_output has the following shape: (batch_size, sequence_length, 768)
          linear1_output = self.linear1(last_hidden_state[:,0,:].view(-1,768)) ## extract the 1st token's embeddings
          # ### New layers:
          linear2_dropout_output = self.dropout1(linear1_output)

          return linear2_dropout_output

retribert_model = CustomBERTModel() # You can pass the parameters if required to have more flexible model
retribert_model.to(torch.device("cuda")) ## can be gpu
criterion = nn.CrossEntropyLoss() ## If required define your own criterion
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, retribert_model.parameters()))

PyTorch code from the instructors (updated to try and combine with my code)

In [21]:
text_pairs = []
for line in range(len(df_train)):
    orig = df_train.iloc[line, 1]
    target = df_train.iloc[line, 2]
    text_pairs.append({'orig': orig, 'target': target})

def preprocess_data(text_pair, tokenizer, max_length=128):
    orig_text, target_text = text_pair
    orig_encoded = tokenizer.batch_encode_plus(
        [orig_text],
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    orig_input_ids = orig_encoded['input_ids'][0]
    orig_attention_mask = orig_encoded['attention_mask'][0]

    target_encoded = float(target_text)
    
    label_ids = target_encoded
    
    return {'input_ids': orig_input_ids,
            'attention_mask': orig_attention_mask,
            'labels': label_ids}

In [25]:
class TranslationDataIterator:
    
    def __init__(self,
                 tokenizer,
                 n_examples,
                 max_load_at_once,
                 data_filename,
                 max_length=128,
                 shuffle=True):
        
        self.tokenizer = tokenizer
        self.n_examples = n_examples
        self.max_load_at_once = max_load_at_once
        self.data_filename = data_filename
        self.max_length = max_length
        self.shuffle = shuffle
        
        # Initialize row order, call on_epoch_end to shuffle row indices
        self.row_order = np.arange(1, self.n_examples+1)
        self.on_epoch_end()

        # Load first chunk of max_load_at_once examples
        self.df_curr_loaded = self._load_next_chunk(0)
        self.curr_idx_in_load = 0
    
    def _load_next_chunk(self, idx):
        load_start = idx
        load_end = idx + self.max_load_at_once

        # Indices to skip are the ones in the shuffled row_order before and
        # after the chunk we'll use for this chunk
        load_idx_skip = self.row_order[:load_start] + self.row_order[load_end:]
        self.df_curr_loaded = pd.read_csv(self.data_filename, skiprows=load_idx_skip)
        self.df_curr_loaded = self.df_curr_loaded.sample(frac=1)
    
    def __len__(self):
        return self.n_examples
    
    def __getitem__(self, idx):
        if self.df_curr_loaded is None or self.curr_idx_in_load >= len(self.df_curr_loaded):
            self._load_next_chunk(idx)
            self.curr_idx_in_load = 0
        
        text_pair = self.df_curr_loaded[['orig', 'target']].values.astype(str)[self.curr_idx_in_load]
        self.curr_idx_in_load += 1
        
        item_data = preprocess_data(
            text_pair,
            self.tokenizer,
            self.max_length
        )
        
        return item_data
    
    def __call__(self):
        for i in range(self.__len__()):
            yield self.__getitem__(i)
            
            if i == self.__len__()-1:
                self.on_epoch_end()
    
    def on_epoch_end(self):
        if self.shuffle:
            self.row_order = list(np.random.permutation(self.row_order))

max_length = 32
max_load_at_once = 100

train_data_iterator = TranslationDataIterator(
    tokenizer=retribert_tokenizer,
    n_examples=len(text_pairs),
    max_load_at_once=max_load_at_once,
    data_filename="train.csv",
    max_length=max_length
)

In [40]:
train_data_iterator.__getitem__

<bound method TranslationDataIterator.__getitem__ of <__main__.TranslationDataIterator object at 0x7f67c1a26e20>>

In [27]:
for epoch in range(5):
  optimizer.zero_grad()
  predicted = []
  outputs_1 = retribert_model(encoded_sent["input_ids"].to("cuda"), encoded_sent["attention_mask"].to("cuda"))
  outputs_1
  train_data_iterator
  #   final_outputs = torch.nn.functional.log_softmax(outputs_1, dim=1)
  # # print(final_outputs)
  #   final_outputs_max = torch.max(final_outputs)
  #   predicted.append(final_outputs_max)
  # print(predicted)
  # print(final_outputs_2)
        # input_ids = encoding['input_ids']
        # attention_mask = encoding['attention_mask']
  # loss = criterion(final_outputs, targets)
        # loss.backward()
        # optimizer.step()

TypeError: ignored

In [ ]:
class My_Custom_Generator(tf.keras.utils.Sequence) :
  
    def __init__(self, images, labels, batch_size) :
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
    
    
    def __len__(self) :
        return (np.ceil(len(self.images) / float(self.batch_size))).astype(np.int)
  
  
    def __getitem__(self, idx) :
        with tf.device('/cpu:0'): 
            batch_x = self.images[idx * self.batch_size : (idx+1) * self.batch_size]
            batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
    
        return batch_x, batch_y

PyTorch code from the instructors

In [ ]:
# with open(text_file) as f:
#     lines = f.read().split('\n')[:-1]

# prefix = 'translate old to modern: '
# text_pairs = []
# for line in lines:
#     orig, target = line.split('\t')
#     text_pairs.append({'orig': prefix + orig, 'target': target})

In [ ]:
# def preprocess_data(text_pair, tokenizer, max_length=128):
#     orig_text, target_text = text_pair
#     orig_encoded = tokenizer.batch_encode_plus(
#         [orig_text],
#         max_length=max_length,
#         padding='max_length',
#         truncation=True,
#         return_attention_mask=True,
#         return_tensors='pt'
#     )

#     orig_input_ids = orig_encoded['input_ids'][0]
#     orig_attention_mask = orig_encoded['attention_mask'][0]
    
#     target_encoded = tokenizer.batch_encode_plus(
#         [target_text],
#         max_length=max_length,
#         padding='max_length',
#         truncation=True,
#         return_attention_mask=True,
#         return_tensors='pt'
#     )
    
#     label_ids = target_encoded['input_ids'][0]
    
#     return {'input_ids': orig_input_ids,
#             'attention_mask': orig_attention_mask,
#             'labels': label_ids}

In [ ]:
# class TranslationDataIterator:
    
#     def __init__(self,
#                  tokenizer,
#                  n_examples,
#                  max_load_at_once,
#                  data_filename,
#                  max_length=128,
#                  shuffle=True):
        
#         self.tokenizer = tokenizer
#         self.n_examples = n_examples
#         self.max_load_at_once = max_load_at_once
#         self.data_filename = data_filename
#         self.max_length = max_length
#         self.shuffle = shuffle
        
#         # Initialize row order, call on_epoch_end to shuffle row indices
#         self.row_order = np.arange(1, self.n_examples+1)
#         self.on_epoch_end()

#         # Load first chunk of max_load_at_once examples
#         self.df_curr_loaded = self._load_next_chunk(0)
#         self.curr_idx_in_load = 0
    
#     def _load_next_chunk(self, idx):
#         load_start = idx
#         load_end = idx + self.max_load_at_once

#         # Indices to skip are the ones in the shuffled row_order before and
#         # after the chunk we'll use for this chunk
#         load_idx_skip = self.row_order[:load_start] + self.row_order[load_end:]
#         self.df_curr_loaded = pd.read_csv(self.data_filename, skiprows=load_idx_skip)
#         self.df_curr_loaded = self.df_curr_loaded.sample(frac=1)
    
#     def __len__(self):
#         return self.n_examples
    
#     def __getitem__(self, idx):
#         if self.df_curr_loaded is None or self.curr_idx_in_load >= len(self.df_curr_loaded):
#             self._load_next_chunk(idx)
#             self.curr_idx_in_load = 0
        
#         text_pair = self.df_curr_loaded[['orig', 'target']].values.astype(str)[self.curr_idx_in_load]
#         self.curr_idx_in_load += 1
        
#         item_data = preprocess_data(
#             text_pair,
#             self.tokenizer,
#             self.max_length
#         )
        
#         return item_data
    
#     def __call__(self):
#         for i in range(self.__len__()):
#             yield self.__getitem__(i)
            
#             if i == self.__len__()-1:
#                 self.on_epoch_end()
    
#     def on_epoch_end(self):
#         if self.shuffle:
#             self.row_order = list(np.random.permutation(self.row_order))

In [ ]:
# # Create the data iterators for train and validation data, pytorch version

# max_length = 32
# max_load_at_once = 100

# train_data_iterator = TranslationDataIterator(
#     tokenizer=t5_tokenizer,
#     n_examples=len(train_pairs),
#     max_load_at_once=max_load_at_once,
#     data_filename=train_file,
#     max_length=max_length
# )

# valid_data_iterator = TranslationDataIterator(
#     tokenizer=t5_tokenizer,
#     n_examples=len(valid_pairs),
#     max_load_at_once=max_load_at_once,
#     data_filename=valid_file,
#     max_length=max_length
# )

# # Specify batch size and other training arguments

# batch_size = 16

# # Modify this filepath to where you want to save the model after fine-tuning
# dir_path = 'drive/MyDrive/ISchool/MIDS/W266/2023_Spring/notebooks/'
# file_path = dir_path + 't5base-finetuned-shakespeare-to-modern'

# args = Seq2SeqTrainingArguments(
#     file_path,
#     evaluation_strategy='epoch',
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     num_train_epochs=1,
# )

# # Define the trainer, passing in the model, training args, and data generators

# trainer = Seq2SeqTrainer(
#     t5_model,
#     args,
#     train_dataset=train_data_iterator,
#     eval_dataset=valid_data_iterator
# )

# trainer.train()

Works but out of memory

In [ ]:
## this seems to be working  but out of memory
for epoch in range(5):
  optimizer.zero_grad()
  predicted = []
  for sent in X_train:
    encoded_sent = retribert_tokenizer.encode_plus(sent, 
                                                   add_special_tokens=True, 
                                                   truncation=True, 
                                                   padding=True, 
                                                   return_attention_mask=True, 
                                                   return_tensors='pt', 
                                                   max_length=510)
    outputs_1 = retribert_model(encoded_sent["input_ids"].to("cuda"), encoded_sent["attention_mask"].to("cuda"))
    outputs_1
    final_outputs = torch.nn.functional.log_softmax(outputs_1, dim=1)
  # # print(final_outputs)
    final_outputs_max = torch.max(final_outputs)
    predicted.append(final_outputs_max)
  # print(predicted)
  # print(final_outputs_2)
        # input_ids = encoding['input_ids']
        # attention_mask = encoding['attention_mask']
  loss = criterion(final_outputs, targets)
  loss.backward()
  optimizer.step()

# Does not work

In [ ]:
# ## trying with dataloader
# # len(X_train_tokens) #3128
# # len(y_train_cohesion) #33128
# cohesion_dict = {"X_train_tokens":X_train_tokens,
#                  "y_train_cohesion": y_train_cohesion}
# cohesion_df = pd.DataFrame(cohesion_dict)

# class customDataset:
#   def __init__(self):
#     xy = cohesion_df
#     self.len = xy.shape[0]
#     self.x_data = torch.from_numpy(xy[:,0:-1])
#     self.y_data = torch.from_numpy(xy.iloc[:,[-1]])
#   def __getitem__(self,index):
#     return self.x_data[index], self.y_data[index]
#   def __len__(self):
#     return self.len

# cohesion_ds = customDataset()
# training_data = torchvision.datasets.cohesion_ds(
#     root="data",
#     train=True,
#     download=True,
#     transform=torchvision.transformsToTensor()
# )

# # train_dataloader = torch.utils.data.DataLoader(cohesion_df, batch_size=8, shuffle=True)
# # train_dataloader

# # train_features, train_labels = iter(train_dataloader)
# # print(f"Feature batch shape: {train_features.size()}")
# # print(f"Labels batch shape: {train_labels.size()}")
# # img = train_features[0].squeeze()
# # label = train_labels[0]
# # plt.imshow(img, cmap="gray")
# # plt.show()
# # print(f"Label: {label}")
# # test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)
## from dataset
class pyDataset(torch.utils.data.Dataset):
  def __init__(self, X,y):
    self.X = X
    self.y = y
  def __len__(self):
    return len(self.X)
  def __getitem__(self,idx):
    return self.X[idx], self.y[idx]
train_loader = torch.utils.data.DataLoader(dataset=pyDataset(X_train, y_train_cohesion), batch_size=8, shuffle=T)

In [ ]:
# for epoch in epochs:
#     for batch in data_loader: ## If you have a DataLoader()  object to get the data.

#         data = batch[0]
#         targets = batch[1] ## assuming that data loader returns a tuple of data and its targets
        
#         optimizer.zero_grad()   
#         encoding = tokenizer.batch_encode_plus(data, return_tensors='pt', padding=True, truncation=True,max_length=510, add_special_tokens = True)
        # outputs = model(input_ids, attention_mask=attention_mask)
        # outputs = F.log_softmax(outputs, dim=1)
        # input_ids = encoding['input_ids']
        # attention_mask = encoding['attention_mask']
        # loss = criterion(outputs, targets)
        # loss.backward()
        # optimizer.step()

In [ ]:
# use MCRMSE as loss metric

# https://stackoverflow.com/questions/61990363/rmse-loss-for-multi-output-regression-problem-in-pytorch
# def loss_function (predicted_x , target):
#     loss = torch.sum(torch.square(predicted_x - target) , axis= 1)/(predicted_x.size()[1])
#     loss = torch.sum(loss)/loss.shape[0]
#     return loss

# loss_fn = nn.MSELoss()
# RMSE_loss = torch.sqrt(loss_fn(prediction, target))
# RMSE_loss.backward()

# def MCRMSE(y_true, y_pred):
#     colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis = 1)
#     return tf.reduce_mean(tf.sqrt(colwise_mse), axis = -1, keepdims = True)
     


In [ ]:
### anothing thing
### https://stackoverflow.com/questions/68115993/input-ids-torch-tensorinput-ids-valueerror-expected-sequence-of-length-133
input_ids = []
attention_masks = []
# print(flaubert)
for sent in X_train:
    encoded_sent = retribert_tokenizer.encode_plus(sent, add_special_tokens=True, truncation=True, padding=True, return_attention_mask=True, return_tensors='pt', max_length=510)

    # Add the outputs to the lists
    # input_ids.append(encoded_sent.get('input_ids'))
    # attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors

# input_ids = torch.as_tensor(input_ids)
# attention_masks = torch.as_tensor(attention_masks)

hidden_state = retribert_model(input_ids=encoded_sent["input_ids"].to("cuda"), attention_mask=encoded_sent["attention_mask"].to("cuda"))
hidden_state
# # # Extract the last hidden state of the token `[CLS]` for classification task
last_hidden_state_cls = hidden_state[0][:, 0, :]

retribert_model.linear1 = nn.Linear(768, 256)
retribert_model.dropout1 = nn.Dropout(0.1)
# device = "cuda" if torch.cuda.is_available() else "cpu"
# device

In [ ]:
list(retribert_model.parameters())[0] # model weights?

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0186, -0.0166, -0.0154,  ...,  0.0220,  0.0171,  0.0076],
        [ 0.0118,  0.0026,  0.0049,  ..., -0.0287,  0.0192,  0.0038],
        ...,
        [-0.0003, -0.0193, -0.0029,  ..., -0.0182, -0.0184, -0.0021],
        [ 0.0355, -0.0214, -0.0174,  ..., -0.0083,  0.0015,  0.0047],
        [-0.0179,  0.0016,  0.0094,  ..., -0.0528,  0.0078,  0.0255]],
       device='cuda:0')

In [ ]:
# Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# # Freeze all base layers in the "features" section of the model (the feature extractor) by setting requires_grad=False
# for param in retribert_model.parameters():
#     param.requires_grad = False

# retribert_model(last_hidden_state_cls)

# ## from https://pypi.org/project/pytorch-pretrained-bert/

# with torch.no_grad():
#     predictions = retribert_model(last_hidden_state_cls)

RuntimeError: ignored

In [ ]:
# results = torch.zeros((len(X_test_tokens), retri_cuda.config.hidden_size)).long()
# results[0]
# with torch.no_grad():
#     for stidx in range(0, len(X_test_tokens), batch_size):
#         X = X_test_tokens[stidx:stidx + batch_size]
#         X = torch.LongTensor(X).cuda()
#         embed, pooled_output = retri_cuda(X)
#         results[stidx:stidx + batch_size,:] = embed.cpu()
# torch.LongTensor(X)

#### from https://machinelearningmastery.com/building-a-regression-model-in-pytorch/
# X_train_torch = torch.tensor(X_train_tokens, dtype=torch.float32)
# y_train_torch = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
# X_test_torch = torch.tensor(X_test, dtype=torch.float32)
# y_test_torch = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)
 
# # training parameters
# n_epochs = 5   # number of epochs to run
# batch_size = 8  # size of each batch
# batch_start = torch.arange(0, len(X_train), batch_size)
 
# # Hold the best model
# best_mse = np.inf   # init to infinity
# best_weights = None
# history = []
 
# # training loop
# for epoch in range(n_epochs):
#     retribert_model.train()
#     with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
#         bar.set_description(f"Epoch {epoch}")
#         for start in bar:
#             # take a batch
#             X_batch = X_train[start:start+batch_size]
#             y_batch = y_train[start:start+batch_size]
#             # forward pass
#             y_pred = retribert_model(X_batch)
#             loss = loss_fn(y_pred, y_batch)
#             # backward pass
#             optimizer.zero_grad()
#             loss.backward()
#             # update weights
#             optimizer.step()
#             # print progress
#             bar.set_postfix(mse=float(loss))
#     # evaluate accuracy at end of each epoch
#     retribert_model.eval()
#     y_pred = retribert_model(X_test)
#     mse = loss_fn(y_pred, y_test)
#     mse = float(mse)
#     history.append(mse)
#     if mse < best_mse:
#         best_mse = mse
#         best_weights = copy.deepcopy(retribert_model.state_dict())
 
# # restore model and return best accuracy
# retribert_model.load_state_dict(best_weights)

In [ ]:
### evaluating the model
# from https://discuss.pytorch.org/t/multi-class-bert-model-class-weights-and-where-to-use-them/176264
# def evaluate():  
#     model.eval()
#     total_loss, total_accuracy = 0, 0
#     total_preds = []
#     for step,batch in enumerate(val_dataloader):
#         if step % 50 == 0 and not step == 0:
#             elapsed = format_time(time.time() - t0)
#             print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))
#         batch = [t.to(device) for t in batch]
#         sent_id, mask, labels = batch
#         with torch.no_grad():
#             preds = model(sent_id, mask)
#             loss = cross_entropy(preds,labels)  #<------- Here
#             total_loss = total_loss + loss.item()
#             preds = preds.detach().cpu().numpy()
#             total_preds.append(preds)
#     avg_loss = total_loss / len(val_dataloader) 
#     total_preds  = np.concatenate(total_preds, axis=0)
#     return avg_loss, total_preds